In [ ]:
# IT/웹/통신 산업의 기업 목록이 총 6204개
    # 1) 개별항목 별 평점은 내비두고, 각 기업의 전체 리뷰 평점만 check하기
    # 1-2) 2차산업군별 {기업명: [평점, 리뷰수, 평균연봉]} <- 3개항목 뽑아내기
        # 1-2-2) 2차산업군별 평점 평균 
        # 1-2-3) 평균연봉과 평점에 상관관계 뽑아낼 수 있을 듯ㅇㅇ.
            # 근데 사실 유의미한 건 아닐듯 -> 연봉 높게 받는 사람이 여기에 불만 글 쓰겠냐..ㅎㅎ;;
            # ? 평균연봉 어케뽑아낸거지 -> 리뷰쓴 사람들의 평균연봉은 아니고, 그냥 임원합친, 전체평균이겠지?
    # 1-3) IT산업 전체 평점 평균 vs 타산업군 평점 평균
    # todo 2) 단, 리뷰가 아예 없어서 나오는 '0'점은 continue
    # 3) 3차분류로, 지역이 있는데 이건 배제 - 다 수도권에 몰려있다는 당연한 사실 빼곤, 뽑을 수 있는 것x

# todo 관건은 페이지네이션 -> 마지막 페이지에 도달했을 때, 어떻게 멈출 것인가?
# select, find, find_all함수 쓰려면 bs이용해서 html파싱작업이 필요하다.
    # 그냥 웹페이지에서 selector copy하고, select(붙이기)하면 안됨
    # 그냥 find_elements쓰면 이 작업 생략되니 이 방식 사용함


In [ ]:
! pip install selenium undetected_chromedriver python-dotenv

In [1]:
import os
import time

from dotenv import load_dotenv
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [ ]:
# 로그인 필요없음
# https://couplewith.tistory.com/428 - webdriverwait.until.visible_locate 코드 해설 굿
# https://www.jobplanet.co.kr/companies?&industry_id=701 -> 다음태그는 'id=702' 
# ----------------

# step0. uc.Chrome()로 크롤링 할 driver객체 만들기
wd = webdriver.Chrome()
second_tags = ['웹에이전시', '포털/인터넷/콘텐츠', '쇼핑몰/오픈마켓', '네트워크/통신/모바일', '하드웨어/장비',
            '보안/백신', '솔루션/SI/ERP/CRM', 'IT컨설팅', '게임', '기타 IT/웹/통신']
urls = [str(x) for x in range(701,711)] # IT의 2차분류

arr = []
for url in urls:
    key = second_tags[int(url)-701]
    wd.get(f'https://www.jobplanet.co.kr/companies?&industry_id={url}')
    time.sleep(3)

    dict, dict2 = {}, {}
    while 1:
        # (1)element's' & (2)section:nth-child(1)~10 -> nth-child(k)를 떼서, section만 남김
        company_infos = wd.find_elements(By.CSS_SELECTOR, '#listCompanies > div > div.section_group > section > div > div > dl.content_col2_4')
        company_names = wd.find_elements(By.CSS_SELECTOR, '#listCompanies > div > div.section_group > section > div > div > dl.content_col2_3.cominfo > dt > a')
        
        now = wd.current_url
        for info, name in zip(company_infos, company_names):
            name = name.text
            
            a = info.text.replace(',','').replace('개의 리뷰','').replace('평균','').replace(' 만원','')
            b = a.split('\n') # ['2507', '평점', '2.8', '4172']
            review, avg_rate, avg_salary = int(b[0]), float(b[2]), int(b[3])

            if review == 0:
                continue
            dict[name] = [review, avg_rate, avg_salary]
        
        # elemen't' - elements하면 x
        wd.find_element(By.CSS_SELECTOR, '#listCompanies > div > div.pg_bottom.um_paginnation > article > a.btn_pgnext').click()
        time.sleep(1)
        
        if now == wd.current_url:
            break
    
    dict2[key] = dict
    arr.append(dict2)
    print(len(dict2[key].keys())) # 누락 데이터 체크용
    
print(arr, len(arr))

# 6204개 데이터 조회 결과: '25분'소요

In [150]:
# 129
# 670
# 189
# 207
# 120
# 31
# 300
# 10
# 311
# 290

{'(주)이스트시큐리티': [96, 3.2, 4243],
 '(주)시큐어원': [90, 3.0, 3221],
 '피앤피시큐어(주)': [80, 3.9, 4352],
 '지니언스(주)': [55, 3.5, 4751],
 '(주)엔에스에이치씨': [46, 2.8, 3653],
 '(주)리얼시큐': [32, 2.9, 2675],
 '킹스정보통신(주)': [32, 3.2, 3319],
 '(주)라바웨이브': [29, 3.9, 3722],
 '(주)씨커스': [28, 3.5, 2503],
 '클라우드브릭(주)': [21, 3.3, 3858],
 '(주)시큐어로그': [15, 4.8, 4670],
 '(주)모카시스템': [14, 4.5, 0],
 '(주)삼문시스템': [12, 1.3, 2743],
 '에스이웍스(주)': [12, 3.8, 4947],
 '(주)에이바이트': [11, 3.1, 3763],
 '시큐레터(주)': [10, 3.3, 4524],
 '피어스넷(주)': [10, 2.5, 2456],
 '(주)스파이스웨어': [7, 1.4, 3837],
 '(주)이지스아이티': [7, 4.7, 2912],
 '(주)코소시스코리아': [7, 2.7, 2625],
 '(주)센스톤': [6, 2.5, 4614],
 '(주)칸나로정보통신': [6, 2.0, 2528],
 '아울시스템즈(주)': [5, 3.4, 3251],
 '아이리노테크(주)': [4, 3.5, 2968],
 '에스에스앤씨(주)': [4, 2.3, 3132],
 '(주)시큐어월': [3, 4.0, 2802],
 '(주)에스투더블유': [3, 5.0, 3765],
 '(주)컴엑스아이': [3, 4.3, 0],
 '(주)테이텀': [2, 4.0, 4230],
 '(주)대진시큐리티': [1, 2.0, 0],
 '(주)유인프라': [1, 2.0, 4188]}

In [153]:
import pandas as pd

english_second_tags = ['web_agency', 'portal_internet_contents', 'shoppingmall_oepnmarket', 'network_commu_mobile', 'hardware_equip',
            'security_vaccine', 'solution_SI_ERP_CRM', 'IT_consulting', 'game', 'etc_IT_web_commu']
column_list = ['리뷰수(개)', '평점', '평균 연봉(만)']

for k in range(len(second_tags)):
    tag = arr[k][second_tags[k]]

    data_for_df = []
    for key, value in tag.items():
        data_for_df.append(value)

    df = pd.DataFrame(data_for_df, index=tag.keys(), columns=column_list)
    
    print(df.tail())

    df.to_csv(f'0-{k}.{english_second_tags[k]}.csv', encoding='utf-8-sig')


# todo 근데, 여기서 또 평균 연봉이 0인게 있음 -> 입력 안한듯 -> NaN인셈 치고, 보안/백신계 평균연봉 넣어주기 걍(결측치 처리)



           리뷰수(개)   평점  평균 연봉(만)
네디아트웍스(주)       1  5.0         0
레드제주            1  4.0         0
스카이디자인(주)       1  5.0         0
스프린트            1  4.0         0
피지엠라이프          1  4.0      2368
               리뷰수(개)   평점  평균 연봉(만)
이라이프커뮤니케이션(주)       1  3.0      2700
카일루아(주)             1  5.0         0
크레실커뮤니케이션           1  2.0         0
플레이플래닛(주)           1  4.0      2298
피플온브릿지              1  5.0         0
          리뷰수(개)   평점  평균 연봉(만)
알파인            1  5.0         0
에이치에이치제이       1  4.0         0
위투쉐어           1  2.0         0
잭슨랩            1  1.0         0
하드워커스          1  3.0         0
            리뷰수(개)   평점  평균 연봉(만)
이노에스제이(주)        1  4.0         0
이노온(주)           1  5.0      2815
이엔에프텔레콤(주)       1  1.0         0
퓨처텔              1  1.0      1575
플랫클(주)           1  2.0      1075
          리뷰수(개)   평점  평균 연봉(만)
(주)파이엇         1  4.0         0
더존넥스트(주)       1  2.0      4516
미래미디어          1  1.0      1260
스마트잉크(주)       1  4.0      4875
큐로이드(주)        1  5.0   

In [ ]:
# pandas, matplotlib로 전처리 -> 시각화